## 메시지 옵션
messages는 대화의 각 부분을 구성하는 데 사용되며, 각 메시지는 "role"과 "content"라는 두 가지 주요 요소를 포함하는 딕셔너리 형태로 되어 있습니다. 
메시지 옵션에 선언되어 있는 각 "role"의 의미는 다음과 같습니다
- "system": 시스템 전역 설정에 대한 지시문을 포함합니다. 예를 들어, 모델에 특정한 페르소나(persona)를 부여하거나, 대화의 맥락을 설정하는 데 사용됩니다.
- "user": 사용자의 입력을 나타냅니다. 이는 대화에서 사용자가 질문하거나 요청한 내용을 담고 있습니다.
- "assistant": AI 모델(예: ChatGPT)의 응답을 나타냅니다. 이는 모델이 생성한 답변이나 정보를 포함합니다.

In [23]:
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()
client = OpenAI()

def ask(question):
    completion = client.chat.completions.create(
        model="gpt-4-turbo-preview",
        messages=[
            {
                "role": "system",
                "content": "너는 친절한 관광 가이드이다. 반드시 한국어로 친절하게 답변해줘.",
            },
            {
                "role": "user",
                "content": question,
            },
        ],
    )
    return completion

**[Quez]** 함수로 선언한 경우, stream 모드를 적용하지 않는 이유는 ???

In [25]:
answers = ask("한국의 수도는?")
print(answers.choices[0].message.content)

한국의 수도는 서울입니다. 서울은 한국의 정치, 경제, 문화의 중심지로 다양한 볼거리와 먹거리, 즐길 거리가 많은 매력적인 도시입니다.


**[Quez]** 이전에 물어본 것에 대해 다시 물어보면??

In [26]:
answers = ask("방금전 물어본 것에 대해 영어로 답변해줘.")
print(answers.choices[0].message.content)

물론이죠. 하지만 제가 확인할 수 있는 이전 대화가 없어서, 어떤 질문에 대한 답변을 원하시는지 구체적으로 알려주실 수 있으실까요? 그러면 거기에 맞는 답변을 제공해 드리겠습니다.


## 대화내용을 기억해두기.(대화의 연속성)
`client.chat.completions.create()` 함수는 자체적으로 대화를 저장하는 기능이 없기 때문에, 
이전 문맥(context)을 기억하면서 대화를 이어나가기 위해서는(일반적인 ChatBot을 생각하시면 됩니다) 
다음과 같이 ask함수에 선언되어 있는 messages 옵션 중 system role에 대해 이전 대화내용을 추가하여 함께 물어봐야 합니다.

In [29]:
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()
client = OpenAI()

def ask(question, history=[]):
    if len(history) == 0:
        # 최초 질문
        history.append(
            {
                "role": "system",
                "content": "You are a helpful assistant. You must answer in Korean.",
            }
        )
    
    # 사용자의 질문을 history에 추가
    history.append(
        {
            "role": "user",
            "content": question
        }
    )
    # history를 물어보면 된다.
    completion = client.chat.completions.create(
        model="gpt-4-turbo-preview",
        messages=history,
    )
    # 마찬가지로 chatGPT 답변을 history에 추가.
    history.append(
        {"role": "assistant", "content": completion.choices[0].message.content}
    )
    return completion

In [31]:
answers = ask("영국의 수도는?")
print(answers.choices[0].message.content)

영국의 수도는 런던입니다.


In [32]:
answers = ask("영어로 말해줘.")
print(answers.choices[0].message.content)

The capital of the United Kingdom is London.
